In [1]:
import numpy as np #importing numpy
import pandas as pd #importing pandas
import PIL #importing pillow for image processing
from PIL import Image#importing image from pillow library
import skimage#importing skimage library
import os#importing os library
import time#importing time library

#Capture start time of the program
start_time = time.time()

In [2]:
from string import digits
#code to remove numbers in a string and assigning it to the variable
r_digits = str.maketrans('', '', digits)
#taking source data folder path to a varible
rawdata_path = 'D://DKIT//dissertation//Dataset_of_Tomato Leaves//plantvillage//Preprocessed_data//32//32_no_bias//'
#changing current librbay to source data
os.chdir(rawdata_path)
#taking sub-directories to a list
dirs = os.listdir()
directories = []
diseases = []
#Taking directories into a list and extract disease name from folder name
for x in dirs:
    res = x.translate(r_digits).lower()
    diseases.append(res)
    temp = rawdata_path + x + '//'
    directories.append(temp)

In [3]:
for i in directories:
    os.chdir(i)
    pics = os.listdir()
    #reading pics one by one in each directory
    for j in pics:
        image = Image.open(j,mode='r')
        size = (32,32)
        image = image.resize(size)
        image = image.save(j)

In [4]:
#creating different dataframes for saving data in different vector forms
#gray data
gray_df = pd.DataFrame(columns=['mean','disease'])
#rgb data
rgb_df = pd.DataFrame(columns=['mean','disease'])
#red data
red_df = pd.DataFrame(columns=['mean','disease'])
#green data
green_df = pd.DataFrame(columns=['mean','disease'])
#blue data
blue_df = pd.DataFrame(columns=['mean','disease'])

In [5]:
#extracting data from images
x = 0
#creating enpty directories to save the vector from in to a list
gray_test = []
rgb_final = []
red_final = []
green_final = []
blue_final = []
disease = []
#reading data directory by directory
for i in directories:
    os.chdir(i)
    k = diseases[x]
    pics = os.listdir()
    #reading pics one by one in each directory
    for j in pics:
        temp_list = []
        rgb_temp = []
        red_temp = []
        green_temp = []
        blue_temp = []
        gray = []
        data = []
        red = []
        green = []
        blue = []
        #opening the image in rgb format
        image = Image.open(j,mode='r')
        data = image.convert('RGB')
        data = np.array(data)
        #extracting only red components
        red = data[:,:,0]
        #extracting only green components
        green = data[:,:,1]
        #extracting only blue components
        blue = data[:,:,2]
        data = data.ravel()
        #saving rgb vector data into list
        rgb_final.append(data)
        rgb_temp.append(np.mean(data))
        rgb_temp.append(k)
        rgb_df.loc[len(rgb_df)] = rgb_temp
        #saving red channel data into dataframe and vector data to a list.
        red = red.ravel()
        red_final.append(red)
        red_temp.append(np.mean(red))
        red_temp.append(k)
        red_df.loc[len(red_df)] = red_temp
        #saving blue channel data into dataframe and vector data to a list.
        blue = blue.ravel()
        blue_final.append(blue)
        blue_temp.append(np.mean(blue))
        blue_temp.append(k)
        blue_df.loc[len(blue_df)] = blue_temp
        #saving green channel data into dataframe and vector data to a list.
        green = green.ravel()
        green_final.append(green)
        green_temp.append(np.mean(green))
        green_temp.append(k)
        green_df.loc[len(green_df)] = green_temp
        #opening the image in gray format
        gray = image.convert('L')
        gray = np.array(gray)
        gray = gray.ravel()
        gray_test.append(gray)
        temp_list.append(np.mean(gray))
        temp_list.append(k)       
        gray_df.loc[len(gray_df)] = temp_list
    x = x+1

In [6]:
os.chdir('D://DKIT//dissertation//Dataset_of_Tomato Leaves//plantvillage//Preprocessed_data//32//32_no_bias_vector//')

In [7]:
#saving the lists as numpy arrays in the local system
np.save('gray_test',gray_test)
np.save('rgb_data',rgb_final)
np.save('red_data',red_final)
np.save('green_data',green_final)
np.save('blue_data',blue_final)

In [8]:
#saving the dataframe as csv files in the local system
gray_df.to_csv('gray_data.csv',index=False)
rgb_df.to_csv('rgb_data.csv',index=False)
red_df.to_csv('red_data.csv',index=False)
green_df.to_csv('green_data.csv',index=False)
blue_df.to_csv('blue_data.csv',index=False)

# KNN on rgb data

In [9]:
#reading numpy array data
rgb_test = np.load('rgb_data.npy')
#reading csv file
rgb_df = pd.read_csv('rgb_data.csv')

In [10]:
loaded = []
for i in rgb_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)

In [11]:
features = loaded
target = rgb_df['disease']
from sklearn.model_selection import train_test_split
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=24)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(Ftrain)
Ftrain = scaler.transform(Ftrain)
Ftest = scaler.transform(Ftest)

In [12]:
from sklearn.neighbors import KNeighborsClassifier
kmodel = KNeighborsClassifier(n_neighbors=75)

In [13]:
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.46403508771929824

# Logistic on rgb data

In [14]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

0.6228070175438597

# SVM on RGB data

In [15]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear')
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.6991228070175438


In [16]:
clf = SVC(decision_function_shape = 'ovo')
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

0.7710526315789473


# Random Forest on RGB 

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5,n_estimators=100, oob_score=True)

In [19]:
classifier_rf.fit(Ftrain, Ltrain)

RandomForestClassifier(max_depth=5, n_jobs=-1, oob_score=True, random_state=42)

In [20]:
classifier_rf.oob_score_

0.5373985523141039

# Naive Bayes on RGB data

In [21]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(Ftrain, Ltrain)

GaussianNB()

In [22]:
y_pred = gnb.predict(Ftest)

In [23]:
from sklearn import metrics
metrics.accuracy_score(Ltest, y_pred)

0.45614035087719296

# KNN on grayscale data

In [24]:
#reading numpy array data
gray_test = np.load('gray_test.npy')
#reading csv file
gray_df = pd.read_csv('gray_data.csv')

In [25]:
loaded = []
for i in gray_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)

In [26]:
features = loaded
target = gray_df['disease']
from sklearn.model_selection import train_test_split
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(Ftrain)
Ftrain = scaler.transform(Ftrain)
Ftest = scaler.transform(Ftest)

In [27]:
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

0.39210526315789473

# Logistic on grayscale data

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='lbfgs', max_iter=10000)
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

# SVM on Grayscale data

In [ ]:
clf = SVC(kernel='linear')
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

# Random Forest on Grayscale

In [ ]:
classifier_rf.fit(Ftrain, Ltrain)

In [ ]:
classifier_rf.oob_score_

# Naive bayes on Grayscale

In [ ]:
gnb.fit(Ftrain, Ltrain)

In [ ]:
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

# KNN on Red channel data

In [ ]:
#reading numpy array data
gray_test = np.load('red_data.npy')
#reading csv file
gray_df = pd.read_csv('red_data.csv')

In [ ]:
loaded = []
for i in gray_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = gray_df['disease']
from sklearn.model_selection import train_test_split
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(Ftrain)
Ftrain = scaler.transform(Ftrain)
Ftest = scaler.transform(Ftest)

In [ ]:
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

# Logistic on Red channel data

In [ ]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

# SVM on Red channel data

In [ ]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

# Random forest on Red channel data

In [ ]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

# Naive Bayes on Red channel data

In [ ]:
gnb.fit(Ftrain, Ltrain)

In [ ]:
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

# KNN on green channel data

In [ ]:
#reading numpy array data
gray_test = np.load('green_data.npy')
#reading csv file
gray_df = pd.read_csv('green_data.csv')
loaded = []
for i in gray_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = gray_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
scaler = StandardScaler()
scaler.fit(Ftrain)
Ftrain = scaler.transform(Ftrain)
Ftest = scaler.transform(Ftest)
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

# Logistic on Green channel data

In [ ]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

# SVM on green channel data

In [ ]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

# Random Forest on green channel data

In [ ]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

# Naive bayes on Green channel data

In [ ]:
gnb.fit(Ftrain, Ltrain)
GaussianNB()
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)

# KNN on blue channel data

In [ ]:
#reading numpy array data
gray_test = np.load('blue_data.npy')
#reading csv file
gray_df = pd.read_csv('blue_data.csv')
loaded = []
for i in gray_test:
    k = []
    k = np.asarray(i,dtype='uint8')
    loaded.append(k)
features = loaded
target = gray_df['disease']
Ftrain,Ftest,Ltrain,Ltest=train_test_split(features,target, test_size=0.20, random_state=20)
scaler = StandardScaler()
scaler.fit(Ftrain)
Ftrain = scaler.transform(Ftrain)
Ftest = scaler.transform(Ftest)
kmodel.fit(Ftrain,Ltrain)
kmodel.score(Ftest,Ltest)

# logistic on blue channel data

In [ ]:
model.fit(Ftrain,Ltrain)
model.score(Ftest,Ltest)

# SVM on blue channel data

In [ ]:
clf.fit(Ftrain,Ltrain)
y_pred = clf.predict(Ftest)
print(accuracy_score(Ltest,y_pred))

# Random Forest on blue channel data

In [ ]:
classifier_rf.fit(Ftrain, Ltrain)
classifier_rf.oob_score_

# Naive bayes on blue channel data

In [ ]:
gnb.fit(Ftrain, Ltrain)
GaussianNB()
y_pred = gnb.predict(Ftest)
metrics.accuracy_score(Ltest, y_pred)